In [1]:
import os, sys
import urllib
import zipfile
import gzip
import pandas as pd

from collections import defaultdict
from goatools.anno.gaf_reader import GafReader
from goatools.obo_parser import GODag

import networkx as nx

data_dir = "../data/"

In [5]:
def download_and_unzip(download_url_link, dir_path, zipped_filename,destination_dir_name, unzip=True):
    #https://www.tutorialsbuddy.com/download-and-unzip-a-zipped-file-in-python
    print("Download starting")

    urllib.request.urlretrieve(
        download_url_link, os.path.join(dir_path, zipped_filename)
    )
    print("Download complete")

    if unzip:
        print("unzipping file starting")
    
        if zipped_filename.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(dir_path, zipped_filename), "r") as zip_file:
                zip_file.extractall(os.path.join(dir_path, destination_dir_name))
        elif zipped_filename.endswith(".gz"):
            print("zipfile")
            with gzip.GzipFile(os.path.join(dir_path, zipped_filename), "rb") as zip_file:
                with open(os.path.join(dir_path, destination_dir_name, zipped_filename.replace(".gz", "")), "wb") as fout:
                    fout.write(zip_file.read())
        else:
            raise NotImplementedError("NO CASE")
            
    
    print("unzipping complete")

In [8]:
reactomeFile = os.path.join(data_dir,"ReactomePathways.gmt")

if not os.path.exists(reactomeFile):
    download_and_unzip("https://reactome.org/download/current/ReactomePathways.gmt.zip", ".", os.path.join(data_dir,"ReactomePathways.gmt.zip"), data_dir)

In [12]:
kg = nx.DiGraph()

with open(reactomeFile) as fin:
    
    for line in fin:
        line = line.strip().split("\t")
        
        reactomeLabel = line[0]
        reactomeID = line[1]
        
        reactomeGenes = [x for x in line[2:] if x[0].isupper()]
               
        if not reactomeID in kg.nodes:
            kg.add_node(reactomeID, id=reactomeID, name=reactomeLabel, score=0)
        
        
        for gene in reactomeGenes:
            if not gene in kg.nodes:
                kg.add_node(gene, type="gene", score=0)
                
            kg.add_edge(gene, reactomeID, type="part_of", score=0)


In [13]:
print(kg)

DiGraph with 13937 nodes and 125897 edges
